In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 5.7 MB/s eta 0:00:00


In [ ]:
import os
import json
from groq import Groq
from datasets import load_dataset
from tqdm import tqdm # A library to show a progress bar

# --- Configuration ---
OUTPUT_FILE = "prepared_data.jsonl"
SAMPLES_TO_GENERATE = 10000
KEYWORDS = [
    # commitments / ownership
    "i will", "i'll", "i can", "i can take", "i can handle", "i'll take", "i'll take this",
    "i'll handle", "i'll own", "i'll do it", "let me handle", "let me take this", "count me in",
    "we will", "we'll", "we can", "we should", "we need to", "someone needs to", "let's",
    "assign", "assigned to", "assign this to", "owner:", "owner is", "set owner",
    "responsible for", "in charge of", "point of contact", "poc",
    "i'll make sure", "i'll ensure", "i'll follow up",

    # directives / requests
    "please", "please make sure", "make sure to", "remember to", "don't forget to",
    "need to", "needs to", "required to", "must", "should", "have to", "expected to",
    "can you", "could you", "would you", "will you",
    "let me know", "keep me posted", "keep us posted",

    # scheduling / coordination
    "schedule", "set up", "setup", "arrange", "organize", "coordinate", "book", "reserve",
    "calendar", "send invite", "send the invite", "send calendar invite", "send meeting invite",
    "reschedule", "postpone", "bring forward", "push back",

    # communication / follow-ups
    "send", "email", "mail", "message", "ping", "dm", "slack", "notify", "inform",
    "reach out", "contact", "call", "follow up", "follow-up", "check in", "circle back",
    "touch base", "get back to", "nudge", "remind",

    # delivery verbs
    "finish", "complete", "deliver", "ship", "release", "submit", "hand over", "handover",
    "prepare", "create", "draft", "write", "document", "summarize",
    "review", "approve", "sign off", "signoff",
    "update", "revise", "refine", "polish", "improve",
    "implement", "execute", "perform", "conduct", "deploy", "roll out", "rollout",
    "configure", "install", "set up", "integrate", "migrate",
    "analyze", "investigate", "triage", "debug", "fix", "patch", "hotfix", "verify",
    "test", "validate", "reproduce", "benchmark",

    # artifacts / deliverables
    "report", "doc", "document", "proposal", "spec", "prd", "design doc",
    "slides", "deck", "presentation", "demo",
    "pr", "pull request", "code review", "cr",
    "issue", "ticket", "task", "story",

    # deadlines / urgency (phrases)
    "deadline", "due", "due date", "asap", "immediately", "urgent", "high priority",
    "priority", "by tomorrow", "by friday", "by monday", "by tuesday", "by wednesday",
    "by thursday", "by saturday", "by sunday",
    "by next week", "by month end", "by eod", "by cob", "by eob", "by eop",
    "end of day", "end of business", "eod", "cob", "eob", "eop",
    "this week", "next week", "this month", "next month",
    "no later than", "latest by", "target date", "target is", "eta",
]

# --- Initialize Groq Client ---

In [ ]:
try:
    client = Groq(api_key="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")
except Exception as e:
    print(f"Error initializing Groq client: {e}")
    print("Please make sure your GROQ_API_KEY is set correctly.")
    exit()

In [ ]:
def generate_action_items_json(dialogue: str) -> str:
    """
    Uses Groq LLM to analyze a dialogue and return a JSON string of action items.
    """
    prompt = f"""
    Analyze the following chat dialogue. Extract all action items.
    Your output MUST be a valid JSON string.
    The JSON should be a list of objects. Each object must have the following keys: "task", "owner", "due_date".
    - "task": A clear and concise description of the task.
    - "owner": The name of the person responsible. If unclear, use "Unassigned".
    - "due_date": The deadline for the task. Use YYYY-MM-DD format if possible. If not, use the text as is (e.g., "next week"). If no due date is mentioned, use "None".

    If there are NO action items in the dialogue, you MUST output an empty JSON list: [].

    Dialogue:
    ---
    {dialogue}
    ---

    JSON Output:
    """
    try:
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="llama-3.1-8b-instant",
            temperature=0.2 # Lower temperature for more deterministic, structured output
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        print(f"\nAn error occurred with Groq API: {e}")
        return "[]" # Return an empty list on error

In [ ]:
print("Loading SAMSum dataset...")
try:
    samsum_dataset = load_dataset("knkarthick/samsum")
    print("Dataset loaded successfully.")
except Exception as e:
    print(f"Failed to load dataset: {e}")
    print("Please ensure you have an internet connection or the dataset is available locally.")
    exit()

Loading SAMSum dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Dataset loaded successfully.


In [ ]:
generated_count = 0
with open(OUTPUT_FILE, 'w') as f:
    # We use tqdm to create a nice progress bar
    for example in tqdm(samsum_dataset['train'], desc="Processing dialogues"):
        dialogue = example['dialogue']

        # Heuristic: Only process dialogues that likely contain action items
        if any(keyword in dialogue.lower() for keyword in KEYWORDS):
            action_json_str = generate_action_items_json(dialogue)

            # Validate that the output is not an empty list and is valid JSON
            try:
                actions = json.loads(action_json_str)
                if isinstance(actions, list) and len(actions) > 0:
                    data_entry = {"dialogue": dialogue, "actions": action_json_str}
                    f.write(json.dumps(data_entry) + "\n")
                    generated_count += 1
            except (json.JSONDecodeError, TypeError):
                # This catches cases where the model returns invalid JSON or something unexpected
                    continue

print(f"\nFinished! Generated {generated_count} samples in '{OUTPUT_FILE}'.")
print("\nNEXT STEP: Manually review and clean the contents of this file.")


Processing dialogues:  34%|███▍      | 1463/4300 [1:28:15<4:38:59,  5.90s/it] 


An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k58yem76fdysafy2165xb7bj` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500207, Requested 446. Please try again in 1m52.976199999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k58yem76fdysafy2165xb7bj` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500207, Requested 257. Please try again in 1m20.283s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1

Processing dialogues:  34%|███▍      | 1471/4300 [1:28:15<1:37:15,  2.06s/it]


An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k58yem76fdysafy2165xb7bj` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500206, Requested 245. Please try again in 1m18.030399999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k58yem76fdysafy2165xb7bj` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500206, Requested 247. Please try again in 1m18.343s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1

Processing dialogues:  35%|███▍      | 1484/4300 [1:28:16<31:48,  1.48it/s]

...


An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k58yem76fdysafy2165xb7bj` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500167, Requested 262. Please try again in 1m14.244999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k58yem76fdysafy2165xb7bj` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500167, Requested 343. Please try again in 1m28.2018s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.

Processing dialogues: 100%|█████████▉| 4293/4300 [2:33:11<00:02,  3.03it/s]


An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k58yem76fdysafy2165xb7bj` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500166, Requested 195. Please try again in 1m2.4454s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k58yem76fdysafy2165xb7bj` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500166, Requested 268. Please try again in 1m15.0208s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-i

Processing dialogues: 100%|██████████| 4300/4300 [2:33:11<00:00,  2.14s/it]


An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k58yem76fdysafy2165xb7bj` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500164, Requested 431. Please try again in 1m42.9752s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k58yem76fdysafy2165xb7bj` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500164, Requested 373. Please try again in 1m32.9038s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

An error occurred with Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-